In [59]:
import pandas as pd
import joblib
from sklearn.preprocessing import LabelEncoder, StandardScaler

# Load dataset again (fresh start in this notebook)
df = pd.read_csv("../data/WA_Fn-UseC_-Telco-Customer-Churn.csv")

# Convert TotalCharges to numeric and drop rows with missing values
df['TotalCharges'] = pd.to_numeric(df['TotalCharges'], errors='coerce')
df = df.dropna(subset=['TotalCharges'])
df.reset_index(drop=True, inplace=True)

print(df.shape)
df.head()


(7032, 21)


,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,...,Yes,No,No,No,One year,No,Mailed check,56.95,1889.50,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,...,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,...,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


In [60]:
df = df.drop('customerID', axis=1)


In [61]:
df_model = df.copy()

# Label Encode binary columns
binary_cols = ['gender','Partner','Dependents','PhoneService','PaperlessBilling','Churn']
le = LabelEncoder()
for col in binary_cols:
    df_model[col] = le.fit_transform(df_model[col])

# One-Hot Encode multi-category columns
df_model = pd.get_dummies(df_model, drop_first=True)

df_model.head()


,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,PaperlessBilling,MonthlyCharges,TotalCharges,Churn,...,TechSupport_Yes,StreamingTV_No internet service,StreamingTV_Yes,StreamingMovies_No internet service,StreamingMovies_Yes,Contract_One year,Contract_Two year,PaymentMethod_Credit card (automatic),PaymentMethod_Electronic check,PaymentMethod_Mailed check
0,0,0,1,0,1,0,1,29.85,29.85,0,...,False,False,False,False,False,False,False,False,True,False
1,1,0,0,0,34,1,0,56.95,1889.50,0,...,False,False,False,False,False,True,False,False,False,True
2,1,0,0,0,2,1,1,53.85,108.15,1,...,False,False,False,False,False,False,False,False,False,True
3,1,0,0,0,45,0,0,42.30,1840.75,0,...,True,False,False,False,False,True,False,False,False,False
4,0,0,0,0,2,1,1,70.70,151.65,1,...,False,False,False,False,False,False,False,False,True,False


In [62]:
# Engineer new features
# TotalServices: Count of all services a customer has
df_model['TotalServices'] = (
    df_model['PhoneService'] + df_model['MultipleLines_Yes'] +
    (df_model['InternetService_Fiber optic'] | df_model['InternetService_No']) +
    df_model['OnlineSecurity_Yes'] + df_model['OnlineBackup_Yes'] +
    df_model['DeviceProtection_Yes'] + df_model['TechSupport_Yes'] +
    df_model['StreamingTV_Yes'] + df_model['StreamingMovies_Yes']
)

# AvgCharges: Monthly charges divided by tenure (a proxy for per-month cost)
df_model['AvgCharges'] = df_model['MonthlyCharges'] / (df_model['tenure'] + 1e-6)

# TotalRevenue: Lifetime value of the customer
df_model['TotalRevenue'] = df_model['tenure'] * df_model['MonthlyCharges']

In [63]:
scaler = StandardScaler()
num_cols = ['tenure','MonthlyCharges','TotalCharges']

df_model[num_cols] = scaler.fit_transform(df_model[num_cols])  

# Update the list of numerical columns to include engineered features
numeric_cols = [
    'tenure', 'MonthlyCharges', 'TotalCharges',
    'TotalServices', 'AvgCharges', 'TotalRevenue'
]

# Scale all the numeric columns
scaler = StandardScaler()
df_model[numeric_cols] = scaler.fit_transform(df_model[numeric_cols])

## Feature Engineering
Here we create new meaningful features from the existing dataset to improve model performance.


In [64]:
# Tenure Groups (categorical bins from tenure)
df_model['TenureGroup'] = pd.cut(
    df['tenure'], 
    bins=[0, 12, 24, 48, 60, 72], 
    labels=['0-12','13-24','25-48','49-60','61-72']
)

In [65]:
# One-hot encode the new TenureGroup feature
df_model = pd.get_dummies(df_model, columns=['TenureGroup'], drop_first=True)

In [66]:
#Total Services Count
# Count how many services a customer has
service_cols = [
    'PhoneService',
    'MultipleLines_Yes',
    'OnlineSecurity_Yes',
    'OnlineBackup_Yes',
    'DeviceProtection_Yes',
    'TechSupport_Yes',
    'StreamingTV_Yes',
    'StreamingMovies_Yes'
]

df_model['TotalServices'] = df_model[service_cols].sum(axis=1)

In [67]:
# Average Charges (Monthly average over tenure)
df_model['AvgCharges'] = df_model['TotalCharges'] / (df['tenure'] + 1)  # +1 avoids division by zero


# Interaction Feature (optional, but useful)
# MonthlyCharges * Tenure → total revenue generated
df_model['TotalRevenue'] = df['MonthlyCharges'] * df['tenure']


In [68]:
print("Final dataset shape:", df_model.shape)
df_model.head()

Final dataset shape: (7032, 38)


,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,PaperlessBilling,MonthlyCharges,TotalCharges,Churn,...,PaymentMethod_Credit card (automatic),PaymentMethod_Electronic check,PaymentMethod_Mailed check,TotalServices,AvgCharges,TotalRevenue,TenureGroup_13-24,TenureGroup_25-48,TenureGroup_49-60,TenureGroup_61-72
0,0,0,1,0,-1.280248,0,1,-1.161694,-0.994194,0,...,False,True,False,1,-0.497097,29.85,False,False,False,False
1,1,0,0,0,0.064303,1,0,-0.260878,-0.173740,0,...,False,False,True,3,-0.004964,1936.30,False,True,False,False
2,1,0,0,0,-1.239504,1,1,-0.363923,-0.959649,1,...,False,False,True,3,-0.319883,107.70,False,False,False,False
3,1,0,0,0,0.512486,0,0,-0.747850,-0.195248,0,...,False,False,False,3,-0.004245,1903.50,False,True,False,False
4,0,0,0,0,-1.239504,1,1,0.196178,-0.940457,1,...,False,True,False,1,-0.313486,141.40,False,False,False,False


Insights from Feature Engineering

Customer Loyalty Segmentation (TenureGroup):

We have grouped customer tenure into distinct categories (e.g., 0-12 months, 13-24 months, etc.). This transformation helps our models identify churn patterns across different stages of a customer's journey.

Expectation: Our analysis of the raw data already suggests that new customers (in the 0-12 months group) will likely have the highest churn rate, while long-term customers (61-72 months) will be the most loyal.

Service Bundling Effect (TotalServices):

A new feature, TotalServices, was created by summing up the number of services each customer subscribes to (Phone, Internet, Streaming, etc.).

Hypothesis: We anticipate that customers who use a wider range of services are more "locked in" to the company and, therefore, less likely to churn. This feature directly tests the impact of service bundling on customer retention.

Spending Habits (AvgCharges):

The AvgCharges feature, calculated as MonthlyCharges divided by TotalServices, provides a granular view of a customer's spending per service.

Hypothesis: This metric helps us identify if customers paying a disproportionately high amount per service are more likely to churn, indicating potential dissatisfaction with pricing.

Customer Lifetime Value (TotalRevenue):

We created TotalRevenue by multiplying Tenure by MonthlyCharges. This is a powerful proxy for a customer's lifetime value.

Insight: High-value customers who are at risk of churning can have a significant financial impact. This feature helps us prioritize retention strategies for the most valuable customer segments.

Standardizing Numerical Features:

We have standardized the numerical features (tenure, MonthlyCharges, TotalCharges, TotalServices, AvgCharges, TotalRevenue) using a StandardScaler. This is a crucial step to ensure that models like Logistic Regression and Random Forest aren't biased by features with larger numerical ranges.

In [69]:
df_model.to_csv('Telco_Churn_Cleaned.csv', index=False)


In [70]:
joblib.dump(scaler, 'scaler.pkl')

['scaler.pkl']